# Deforestation samples labelling tool
gemasakti.adzan@wri.org

In [ ]:
# packages
import ee
from IPython.display import display, clear_output
import geopandas
import glob
import ipyleaflet
import ipywidgets as widgets
import numpy
import pandas

In [ ]:
# ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
layout = widgets.Layout(width = 'auto')
user_name = widgets.Text(
    value = None,
    placeholder = 'Tuliskan nama anda!',
    description = 'User name',
    disabled = False,
    layout = layout,
    style = {'description_width': 'initial'}
)
sample_group = widgets.Text(
    value = None,
    placeholder = 'Tuliskan A atau B sesuai kelompok Anda!',
    description = 'Sample group',
    disabled = False,
    layout = layout,
    style = {'description_width': 'initial'}
)
mapping_area = widgets.Dropdown(
    options = ['Papua', 'Papua_Barat'],
    value = None,
    placeholder = 'Pilih salah satu provinsi!',
    description = 'Province',
    layout = layout,
    style = {'description_width': 'initial'}
)
deforestation_year = widgets.Dropdown(
    options = ['2016','2017','2018','2019','2020','2021'],
    value = '2021',
    description = 'Deforestation year',
    layout = layout,
    style = {'description_width': 'initial'}
)

In [ ]:
idx = 0

def main_fun(user_text, defo_year, group):
    
    year = defo_year
    username = user_text
    group_sample = group
    sample_file = 'samples/' + year + '/TCL_' + year + '_Samples_Papua_Barat_' + group_sample + '_4326.shp'
    
    gdf = geopandas.read_file(sample_file)
    # create some columns for first time only
    if 'ID' in gdf:
        pass
    else:
        gdf['Username'] = username
        gdf['ID'] = numpy.arange(len(gdf))
        gdf['lat'] = gdf['geometry'].y
        gdf['lon'] = gdf['geometry'].x
        gdf['Class'] = None

    # create list of ID
    id_list = gdf['ID'].tolist()
    
    # Create list of landcover class name
    class_text = 'Tutupan kanopi pohon (false detection), Alami (erosi; longsor; banjir; meandering; dsb.), Logging, Hutan tanaman, Pertanian, Perkebunan kelapa sawit, Perkebunan lainnya, Pertambangan, Pembangunan infrastruktur (lahan terbangun; jaringan jalan;  dsb.), Tidak yakin'
    class_list = class_text.split(',')

    # ee.Image
    aoi = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level1').filter(ee.Filter.inList('ADM1_NAME', ['Papua','Papua Barat'])).geometry().bounds()
    img = ee.Image('users/gemasaktiadzan/Planet_Papua_Barat_2022')
    tcl_id = 'users/gemasaktiadzan/GFC_1_9_Tanah_Papua_TCL_' + year
    gfw = ee.Image(tcl_id)
    band_names = img.bandNames().getInfo()
    
    # basemaps
    basemap_Esri = ipyleaflet.basemap_to_tiles(basemap = ipyleaflet.basemaps.Esri.WorldImagery)
    hph = geopandas.read_file('layers/2020_HA_PB.shp')
    ilok = geopandas.read_file('layers/2020_ILOK_PB.shp')
    
    # Ipyleaflet components
    m = ipyleaflet.Map(scroll_wheel_zoom = True, attribution_control = True)
    layer_control = ipyleaflet.LayersControl(position = 'topright')
    m.add_control(layer_control)
    
    # Data labelling toolbar widgets
    out = widgets.Output()
    next_button = widgets.Button(
        description = 'Next feature',
        button_style = 'info',
        tooltip = 'Next feature',
        icon = 'toggle-right',
        layout = {'width': '200px'})
    prev_button = widgets.Button(
        description = 'Prev. feature',
        button_style = 'info',
        tooltip = 'Prev. feature',
        icon = 'toggle-left',
        layout = {'width': '200px'})
    jump_button = widgets.Button(
        description = 'Jump to feature',
        button_style = 'info',
        tooltip = 'Jump to',
        icon = 'mail-forward',
        layout = {'width': '200px'})
    class_names = widgets.Dropdown(
        options = class_list,
        description = 'Class name:',
        value = class_list[0],
        layout = {'width': '400px'})
    id_selector = widgets.IntText(
        description = "ID",
        value = id_list[0],
        layout = {'width': '200px'})
    toolbar_wid = widgets.VBox()
    toolbar_wid.children = (
        out,
        widgets.HBox([id_selector,jump_button]),
        widgets.HBox([prev_button,next_button])
    )
    
    def show_map():
        
        global idx
        global geodata
        
        def GetTileLayerUrl(ee_image_object):
            map_id = ee.Image(ee_image_object).getMapId()
            tile_fetcher = map_id['tile_fetcher']
            return tile_fetcher.url_format

        # layer_name_split = asset_id_text.split('/')[-1]
        planet_image = ipyleaflet.TileLayer(url = GetTileLayerUrl(img.visualize(min = 150, max = 1200, gamma = 0.99, bands = ['R','G','B'])))
        planet_group = ipyleaflet.LayerGroup(name = "Planet June 2021-June 2022", layer = planet_image)
        gfw_image = ipyleaflet.TileLayer(url = GetTileLayerUrl(gfw.visualize(min = 1, max = 1, palette = 'red')))
        gfw_group = ipyleaflet.LayerGroup(name = "Tree cover loss 2021", layer = gfw_image)
        m.clear_layers()
        # m.add_layer(basemap_Esri)
        m.add_layer(planet_group)
        m.add_layer(gfw_group)
        planet_group.add_layer(planet_image)
        gfw_group.add_layer(gfw_image)
        
        init_obj = gdf[gdf['ID'] == id_list[idx]]
        geodata = ipyleaflet.GeoData(geo_dataframe = init_obj, name = 'Sample ' + str(id_list[idx]), point_style = {'radius': 5, 'color': None, 'fillOpacity': 1, 'fillColor': 'blue', 'weight': 3})
        hph_con = ipyleaflet.GeoData(geo_dataframe = hph, name = 'Konsesi HPH',
                                    style = {'fillColor': 'Red', 'color': 'Red'})
        ilok_con = ipyleaflet.GeoData(geo_dataframe = ilok, name = 'Izin lokasi perkebunan',
                                    style = {'fillColor': 'Blue', 'color': 'Blue'})
        m.add_layer(hph_con)
        m.add_layer(ilok_con)
        lat = init_obj.iloc[0]['lat']
        lon = init_obj.iloc[0]['lon']
        m.center = (lat, lon)
        m.zoom = 10
        m.add_layer(geodata)
        
        def assign_class():
            
            global idx
            
            def get_idx(id):
                return id_list.index(id)
            
            def jump_button_clicked(b):
                with out:
                    clear_output()
                    global idx
                    global geodata
                    idx = get_idx(id_selector.value)
                    print('jump to ID:', id_selector.value)
                    m.remove_layer(geodata)
                    geo_dataframe = gdf.loc[gdf['ID'] == id_list[idx]]
                    geodata = ipyleaflet.GeoData(geo_dataframe = geo_dataframe, name = 'Sample ' + str(id_list[idx]),point_style = {'radius': 5, 'color': None, 'fillOpacity': 1, 'fillColor': 'blue', 'weight': 3})
                    m.add_layer(geodata)
                    sel = gdf.loc[gdf['ID'] == id_list[idx]]
                    lat = sel.iloc[0]['lat']
                    lon = sel.iloc[0]['lon']
                    m.center = (lat, lon)

            def next_button_clicked(b, incr = 1):
                with out:
                    clear_output()
                    global idx
                    global geodata
                    idx = idx + incr
                    m.remove_layer(geodata)
                    geo_dataframe = gdf.loc[gdf['ID'] == id_list[idx]]
                    geodata = ipyleaflet.GeoData(geo_dataframe = geo_dataframe, name = 'Layer ' + str(id_list[idx]),
                                                point_style = {'radius': 5, 'color': None, 'fillOpacity': 1, 'fillColor': 'blue', 'weight': 3})
                    print('Current ID Class:', geo_dataframe['Class'])
                    m.add_layer(geodata)
                    sel = gdf.loc[gdf['ID'] == id_list[idx]]
                    lat = sel.iloc[0]['lat']
                    lon = sel.iloc[0]['lon']
                    m.center = (lat, lon)
            
            def prev_button_clicked(b):
                return next_button_clicked(b, -1)
            
            jump_button.on_click(jump_button_clicked)
            next_button.on_click(next_button_clicked)
            prev_button.on_click(prev_button_clicked)
            
            def assign_f(classname):
                global idx
                gdf.loc[gdf['ID'] == id_list[idx], 'Class'] = classname
            
            def to_file():
                gdf.to_file(sample_file)
        
            display(m)
            display(toolbar_wid)
            
            im = widgets.interact_manual(assign_f, classname = class_names, gdf = widgets.fixed(gdf))
            im.widget.children[0].description = 'Class name:'
            im.widget.children[1].description = 'Assign class!'
            
            sm = widgets.interact_manual(to_file)
            sm.widget.children[0].description = 'Save edit to shapefiles'

        assign_class()
    
    print('Show map!')
    widgets.interact_manual(show_map)

print('Dataset parameter settings:')
widgets.interact_manual(main_fun, 
                        user_text = user_name,
                        defo_year = deforestation_year,
                        group = sample_group)